# KRules Framework - Complete Showcase

This notebook demonstrates all major features of KRules Framework 3.0.

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Basic Concepts](#basic-concepts)
4. [Subjects & Properties](#subjects-properties)
5. [Lambda Values (Atomic Operations)](#lambda-values)
6. [Event Handlers](#event-handlers)
7. [Filters](#filters)
8. [Middleware](#middleware)
9. [Property Change Events](#property-changes)
10. [Extra Context](#extra-context)
11. [Extended Properties](#extended-properties)
12. [Storage Backends](#storage-backends)
13. [Complete Example](#complete-example)

## Introduction {#introduction}

KRules is an **async-first event-driven framework** for Python applications.

**Key Features:**
- Reactive property store with automatic event emission
- Decorator-based handler system
- Pluggable storage backends (in-memory, Redis, PostgreSQL)
- Container-based dependency injection
- Type-safe, modern Python (3.11+)

**Core Concepts:**
- **Subjects** - Dynamic entities with reactive state
- **Event Bus** - Routes events to matching handlers
- **Handlers** - React to events with optional filters
- **Middleware** - Cross-cutting concerns (logging, timing, etc.)

## Setup {#setup}

Install KRules Framework:

In [29]:
# Installation (run once)
# !pip install krules-framework

# For this notebook, we assume krules is already installed
import asyncio
from datetime import datetime
from krules_core.container import KRulesContainer

print("✓ KRules Framework loaded successfully")

✓ KRules Framework loaded successfully


## Basic Concepts {#basic-concepts}

KRules uses a **container** to manage all components:

In [2]:
# Create container
container = KRulesContainer()

# Get handler decorators and emit function
on, when, middleware, emit = container.handlers()

print("✓ Container created")
print("✓ Handlers ready:", type(on).__name__, type(when).__name__, type(middleware).__name__)

✓ Container created
✓ Handlers ready: function function function


## Subjects & Properties {#subjects-properties}

**Subjects** are dynamic entities with reactive properties.

### Creating Subjects

In [3]:
# Create subjects
user = container.subject("user-123")
device = container.subject("device-456")
order = container.subject("order-789")

print(f"Created subjects: {user}, {device}, {order}")

Created subjects: user-123, device-456, order-789


### Setting Properties

In [4]:
# Set simple properties
await user.set("name", "John Doe")
await user.set("email", "john@example.com")
await user.set("age", 30)

# Set complex properties
await user.set("address", {
    "street": "123 Main St",
    "city": "Boston",
    "zip": "02101"
})

await user.set("tags", ["premium", "verified"])

print("✓ Properties set")
print(f"Name: {await user.get('name')}")
print(f"Email: {await user.get('email')}")
print(f"Age: {await user.get('age')}")
print(f"Address: {await user.get('address')}")
print(f"Tags: {await user.get('tags')}")

✓ Properties set
Name: John Doe
Email: john@example.com
Age: 30
Address: {'street': '123 Main St', 'city': 'Boston', 'zip': '02101'}
Tags: ['premium', 'verified']


### Getting Properties

In [5]:
# Get property
name = await user.get("name")
print(f"Name: {name}")

# Get with default value
status = await user.get("status", default="inactive")
print(f"Status: {status}")

# Check existence
has_email = await user.has("email")
has_phone = await user.has("phone")
print(f"Has email: {has_email}, Has phone: {has_phone}")

# List all properties
keys = await user.keys()
print(f"All properties: {keys}")

Name: John Doe
Status: inactive
Has email: True, Has phone: False
All properties: ['name', 'email', 'age', 'address', 'tags']


## Lambda Values (Atomic Operations) {#lambda-values}

Use lambda functions for atomic updates:

In [6]:
# Counter pattern - atomic increment
counter = container.subject("counter-1")
await counter.set("count", 0)

print(f"Initial count: {await counter.get('count')}")

# Increment atomically
await counter.set("count", lambda c: c + 1)
await counter.set("count", lambda c: c + 1)
await counter.set("count", lambda c: c + 1)

print(f"After 3 increments: {await counter.get('count')}")

# Atomic list append
await user.set("login_history", [])
await user.set("login_history", lambda hist: hist + [datetime.now().isoformat()])
await user.set("login_history", lambda hist: hist + [datetime.now().isoformat()])

print(f"Login history: {await user.get('login_history')}")

# Conditional update
await user.set("high_score", 100)
await user.set("high_score", lambda current: max(current, 150))  # Update to 150
await user.set("high_score", lambda current: max(current, 120))  # No change (150 > 120)

print(f"High score: {await user.get('high_score')}")

Initial count: 0
After 3 increments: 3
Login history: ['2025-11-10T10:26:51.453277', '2025-11-10T10:26:51.453347']
High score: 150


## Event Handlers {#event-handlers}

Handlers react to events using the `@on` decorator:

In [7]:
# Reset container for clean handler registration
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# Simple handler
@on("user.login")
async def handle_login(ctx):
    print(f"[Handler] User logged in: {ctx.subject.name}")
    await ctx.subject.set("last_login", datetime.now().isoformat())

# Multiple event patterns
@on("user.created", "user.updated")
async def handle_user_change(ctx):
    print(f"[Handler] User changed: {ctx.event_type}")

# Glob patterns
@on("device.*")
async def handle_device_events(ctx):
    print(f"[Handler] Device event: {ctx.event_type}")

# Wildcard (all events)
@on("*")
async def log_all_events(ctx):
    print(f"[Logger] Event: {ctx.event_type} on {ctx.subject.name}")

# Test handlers
user = container.subject("user-123")
await emit("user.login", user, {})
await emit("user.created", user, {})
await emit("device.connected", container.subject("device-1"), {})

[Handler] User logged in: user-123
[Logger] Event: subject-property-changed on user-123
[Logger] Event: user.login on user-123
[Handler] User changed: user.created
[Logger] Event: user.created on user-123
[Handler] Device event: device.connected
[Logger] Event: device.connected on device-1


## Filters {#filters}

Use `@when` to add conditional execution:

In [9]:
# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# Handler with filter
@on("order.created")
@when(lambda ctx: ctx.payload.get("amount", 0) > 1000)
async def handle_large_order(ctx):
    print(f"[Handler] Large order: ${ctx.payload['amount']}")
    # ctx.emit uses current subject by default
    await ctx.emit("order.requires_approval", ctx.payload)

# Multiple filters (AND logic)
@on("user.action")
@when(lambda ctx: ctx.payload.get("action") == "purchase")
@when(lambda ctx: ctx.payload.get("amount", 0) > 500)
async def handle_large_purchase(ctx):
    print(f"[Handler] Large purchase: ${ctx.payload['amount']}")

# Test filters
order1 = container.subject("order-1")
order2 = container.subject("order-2")

print("\nSmall order (no handler triggered):")
await emit("order.created", order1, {"amount": 500})

print("\nLarge order (handler triggered):")
await emit("order.created", order2, {"amount": 1500})

print("\nLarge purchase (both filters pass):")
user = container.subject("user-1")
await emit("user.action", user, {"action": "purchase", "amount": 600})


Small order (no handler triggered):

Large order (handler triggered):
[Handler] Large order: $1500

Large purchase (both filters pass):
[Handler] Large purchase: $600


## Middleware {#middleware}

Middleware intercepts all events:

In [10]:
# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

import time

# Logging middleware
@middleware
async def logging_middleware(ctx, next):
    print(f"[Middleware] Before: {ctx.event_type}")
    await next()
    print(f"[Middleware] After: {ctx.event_type}")

# Timing middleware
@middleware
async def timing_middleware(ctx, next):
    start = time.time()
    await next()
    duration = time.time() - start
    print(f"[Timing] {ctx.event_type} took {duration*1000:.2f}ms")

# Handler
@on("test.event")
async def test_handler(ctx):
    print(f"[Handler] Processing {ctx.event_type}")
    await asyncio.sleep(0.01)  # Simulate work

# Test middleware
test_subject = container.subject("test")
await emit("test.event", test_subject, {})

[Middleware] Before: test.event
[Handler] Processing test.event
[Timing] test.event took 10.89ms
[Middleware] After: test.event


## Property Change Events {#property-changes}

Property changes automatically emit `subject-property-changed` events:

In [13]:
# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# React to property changes
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status")
async def on_status_change(ctx):
    print(f"[Handler] Status changed: {ctx.old_value} → {ctx.new_value}")
    
    if ctx.new_value == "active":
        # ctx.emit uses current subject by default
        await ctx.emit("user.activated", {})

# React to any property change
@on("subject-property-changed")
async def audit_trail(ctx):
    print(f"[Audit] {ctx.subject.name}.{ctx.property_name} = {ctx.new_value}")

# Test property changes
user = container.subject("user-123")
await user.set("status", "pending")
await user.set("status", "active")
await user.set("email", "john@example.com")

[Handler] Status changed: None → pending
[Audit] user-123.status = pending
[Handler] Status changed: pending → active
[Audit] user-123.status = active
[Audit] user-123.email = john@example.com


('john@example.com', None)

## Extra Context {#extra-context}

Pass additional context to handlers using the `extra` parameter:

In [14]:
# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# Handler that uses extra context
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status")
async def on_status_change_with_context(ctx):
    if ctx.extra:
        reason = ctx.extra.get("reason", "unknown")
        admin = ctx.extra.get("admin_id", "system")
        print(f"[Handler] Status changed to {ctx.new_value}")
        print(f"  Reason: {reason}")
        print(f"  Changed by: {admin}")
    else:
        print(f"[Handler] Status changed to {ctx.new_value} (no extra context)")

# Set property with extra context
user = container.subject("user-123")

print("\nWithout extra context:")
await user.set("status", "pending")

print("\nWith extra context:")
await user.set("status", "suspended", extra={
    "reason": "policy_violation",
    "admin_id": "admin-456",
    "timestamp": datetime.now().isoformat()
})

# Delete with extra context
@on("subject-property-deleted")
async def on_property_deleted(ctx):
    reason = (ctx.extra or {}).get("reason", "unknown")
    print(f"[Handler] Property {ctx.property_name} deleted (reason: {reason})")

await user.set("temp_data", "value")
await user.delete("temp_data", extra={"reason": "expired"})


Without extra context:
[Handler] Status changed to pending (no extra context)

With extra context:
[Handler] Status changed to suspended
  Reason: policy_violation
  Changed by: admin-456
[Handler] Property temp_data deleted (reason: expired)


## Extended Properties {#extended-properties}

Extended properties store metadata without emitting events:

In [15]:
# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# Track event emissions
event_count = [0]

@on("subject-property-changed")
async def count_events(ctx):
    event_count[0] += 1

user = container.subject("user-123")

# Normal property (emits event)
await user.set("name", "John")
print(f"After normal set: {event_count[0]} event(s)")

# Extended property (no event)
await user.set_ext("last_ip", "192.168.1.1")
await user.set_ext("user_agent", "Mozilla/5.0")
print(f"After extended sets: {event_count[0]} event(s) (still same)")

# Get extended properties
ip = await user.get_ext("last_ip")
print(f"\nLast IP: {ip}")

# Get all extended properties
ext_props = await user.get_ext_props()
print(f"All extended properties: {ext_props}")

# Export to dict (includes both types)
data = await user.dict()
print(f"\nFull export: {data}")

After normal set: 1 event(s)
After extended sets: 1 event(s) (still same)

Last IP: 192.168.1.1
All extended properties: {'last_ip': '192.168.1.1', 'user_agent': 'Mozilla/5.0'}

Full export: {'name': 'John', 'ext': {'last_ip': '192.168.1.1', 'user_agent': 'Mozilla/5.0'}}


## Storage Backends {#storage-backends}

KRules supports multiple storage backends:

### 1. EmptySubjectStorage (In-Memory)

Default storage - no persistence:

In [16]:
# Default container uses EmptySubjectStorage
container = KRulesContainer()

user = container.subject("user-memory")
await user.set("name", "John")
await user.set("email", "john@example.com")

print(f"Name: {await user.get('name')}")
print(f"Email: {await user.get('email')}")
print("\n⚠️  Data is in-memory only - lost when process ends")

Name: John
Email: john@example.com

⚠️  Data is in-memory only - lost when process ends


### 2. Redis Storage

Persistent storage with Redis:

In [26]:
# Redis Storage Example
# Requires: pip install "krules-framework[redis]"

try:
    from dependency_injector import providers
    from redis.asyncio import Redis
    from redis_subjects_storage.storage_impl import create_redis_storage
    
    # Create Redis container
    redis_container = KRulesContainer()
    
    # Create Redis client (without decode_responses - storage expects bytes)
    redis_client = Redis.from_url("redis://localhost:6379")
    
    # Create storage factory
    redis_storage_factory = create_redis_storage(
        redis_client=redis_client,
        redis_prefix="krules:showcase:"
    )
    
    # Override storage provider
    redis_container.subject_storage.override(
        providers.Object(redis_storage_factory)
    )
    
    print("=== Redis Storage Demo ===\n")
    
    # 1. Create and persist subject
    print("1. Creating new user...")
    user = redis_container.subject("user-redis-123")
    await user.set("name", "Alice")
    await user.set("email", "alice@example.com")
    await user.set("role", "admin")
    await user.set("login_count", 0)
    await user.store()  # Persist to Redis
    print(f"   ✓ User created: {await user.get('name')} ({await user.get('role')})")
    
    # 2. Load existing subject and modify
    print("\n2. Loading existing user and updating...")
    user2 = redis_container.subject("user-redis-123")
    current_name = await user2.get("name")
    print(f"   Loaded: {current_name}")
    
    # Update properties
    await user2.set("role", "superadmin")
    await user2.set("last_login", datetime.now().isoformat())
    await user2.store()
    print(f"   ✓ Updated role to: {await user2.get('role')}")
    
    # 3. Atomic operation with lambda
    print("\n3. Atomic increment (simulating multiple logins)...")
    for i in range(5):
        user3 = redis_container.subject("user-redis-123")
        await user3.set("login_count", lambda c: c + 1)
        await user3.store()
    
    user_final = redis_container.subject("user-redis-123")
    login_count = await user_final.get("login_count")
    print(f"   ✓ Login count after 5 increments: {login_count}")
    
    # 4. Verify full state persistence
    print("\n4. Full state verification...")
    user_verify = redis_container.subject("user-redis-123")
    data = await user_verify.dict()
    print(f"   Complete state: {data}")
    
    # 5. Complex atomic operation
    print("\n5. Atomic list append...")
    await user_verify.set("login_history", [])
    await user_verify.store()
    
    for i in range(3):
        user_temp = redis_container.subject("user-redis-123")
        await user_temp.set("login_history", lambda hist: hist + [datetime.now().isoformat()])
        await user_temp.store()
    
    user_history = redis_container.subject("user-redis-123")
    history = await user_history.get("login_history")
    print(f"   ✓ Login history entries: {len(history)}")
    
    print("\n✓ Redis storage demo completed successfully!")
    
    # Cleanup
    await redis_client.aclose()
    
except ImportError:
    print("⚠️  Redis storage not available")
    print("   Install with: pip install 'krules-framework[redis]'")
except Exception as e:
    print(f"⚠️  Redis connection failed: {e}")
    print("   Make sure Redis is running on localhost:6379")

=== Redis Storage Demo ===

1. Creating new user...
   ✓ User created: Alice (admin)

2. Loading existing user and updating...
   Loaded: Alice
   ✓ Updated role to: superadmin

3. Atomic increment (simulating multiple logins)...
   ✓ Login count after 5 increments: 5

4. Full state verification...
   Complete state: {'name': 'Alice', 'login_count': 5, 'email': 'alice@example.com', 'role': 'superadmin', 'last_login': '2025-11-10T11:09:45.265407', 'ext': {}}

5. Atomic list append...
   ✓ Login history entries: 3

✓ Redis storage demo completed successfully!


### 3. PostgreSQL Storage

Persistent storage with PostgreSQL + JSONB:

**Prerequisites:**
1. PostgreSQL server running
2. Database created: `createdb krules` (or `CREATE DATABASE krules;` in psql)
3. User with access to the database

**Schema auto-creation:** Tables and indexes are created automatically on first use.

**Configuration:** Adjust connection parameters in the code below to match your PostgreSQL setup.

In [30]:
# PostgreSQL Storage Example
# Requires: pip install "krules-framework[postgres]"

try:
    from dependency_injector import providers
    import asyncpg
    from postgres_subjects_storage.storage_impl import create_postgres_storage
    
    # Create PostgreSQL container
    pg_container = KRulesContainer()
    
    # Create PostgreSQL connection pool
    # IMPORTANT: Adjust these parameters for your environment
    pg_pool = await asyncpg.create_pool(
        database="krules",          # Database name
        user="postgres",            # Your PostgreSQL username
        password="postgres",        # Your PostgreSQL password (or None/omit if not needed)
        host="localhost",           # PostgreSQL host
        port=5432                   # PostgreSQL port
    )
    
    # Create storage factory
    pg_storage_factory = create_postgres_storage(pool=pg_pool)
    
    # Override storage provider
    pg_container.subject_storage.override(
        providers.Object(pg_storage_factory)
    )
    
    print("=== PostgreSQL Storage Demo ===\n")
    
    # 1. Create and persist subject
    print("1. Creating new employee...")
    employee = pg_container.subject("employee-pg-456")
    await employee.set("name", "Bob Smith")
    await employee.set("email", "bob@company.com")
    await employee.set("department", "Engineering")
    await employee.set("salary", 75000)
    await employee.set("projects", ["project-A", "project-B"])
    await employee.set("metadata", {
        "level": "Senior",
        "location": "Remote"
    })
    await employee.store()  # Persist to PostgreSQL
    print(f"   ✓ Employee created: {await employee.get('name')}")
    
    # 2. Load existing subject and modify
    print("\n2. Loading existing employee and updating...")
    emp2 = pg_container.subject("employee-pg-456")
    current_dept = await emp2.get("department")
    print(f"   Current department: {current_dept}")
    
    # Update properties
    await emp2.set("department", "Platform Engineering")
    await emp2.set("last_review", datetime.now().isoformat())
    await emp2.store()
    print(f"   ✓ Updated department to: {await emp2.get('department')}")
    
    # 3. Atomic salary increase
    print("\n3. Atomic salary increase (5% raise)...")
    emp3 = pg_container.subject("employee-pg-456")
    # .set() returns (new_value, old_value)
    new_salary, old_salary = await emp3.set("salary", lambda current: int(current * 1.05))
    await emp3.store()
    print(f"   ✓ Salary: ${old_salary:,} → ${new_salary:,}")
    
    # 4. Atomic list operations
    print("\n4. Adding projects atomically...")
    for project in ["project-C", "project-D"]:
        emp_temp = pg_container.subject("employee-pg-456")
        await emp_temp.set("projects", lambda proj_list: proj_list + [project])
        await emp_temp.store()
    
    emp_projects = pg_container.subject("employee-pg-456")
    projects = await emp_projects.get("projects")
    print(f"   ✓ Total projects: {len(projects)} - {projects}")
    
    # 5. Complex nested object update
    print("\n5. Updating nested metadata...")
    emp_meta = pg_container.subject("employee-pg-456")
    await emp_meta.set("metadata", lambda meta: {
        **meta,
        "level": "Staff",  # Promotion!
        "certifications": ["GCP", "GKE"]
    })
    await emp_meta.store()
    metadata = await emp_meta.get("metadata")
    print(f"   ✓ Updated metadata: {metadata}")
    
    # 6. Verify full persistence across connections
    print("\n6. Full state verification (new connection)...")
    emp_verify = pg_container.subject("employee-pg-456")
    data = await emp_verify.dict()
    print(f"   Name: {data.get('name')}")
    print(f"   Department: {data.get('department')}")
    print(f"   Salary: ${data.get('salary'):,}")
    print(f"   Projects: {len(data.get('projects', []))} active")
    
    print("\n✓ PostgreSQL storage demo completed successfully!")
    
    # Cleanup
    await pg_pool.close()
    
except ImportError:
    print("⚠️  PostgreSQL storage not available")
    print("   Install with: pip install 'krules-framework[postgres]'")
except Exception as e:
    print(f"⚠️  PostgreSQL connection failed: {e}")
    print("   Check connection parameters (user, password, database, host, port)")

=== PostgreSQL Storage Demo ===

1. Creating new employee...
   ✓ Employee created: Bob Smith

2. Loading existing employee and updating...
   Current department: Engineering
   ✓ Updated department to: Platform Engineering

3. Atomic salary increase (5% raise)...
   ✓ Salary: $75,000 → $78,750

4. Adding projects atomically...
   ✓ Total projects: 4 - ['project-A', 'project-B', 'project-C', 'project-D']

5. Updating nested metadata...
   ✓ Updated metadata: {'level': 'Staff', 'location': 'Remote', 'certifications': ['GCP', 'GKE']}

6. Full state verification (new connection)...
   Name: Bob Smith
   Department: Platform Engineering
   Salary: $78,750
   Projects: 4 active

✓ PostgreSQL storage demo completed successfully!


## Complete Example {#complete-example}

A complete example demonstrating multiple features together:

In [ ]:
# Complete Example: Order Processing System

# Reset container
container = KRulesContainer()
on, when, middleware, emit = container.handlers()

# Middleware: Logging
@middleware
async def logging_middleware(ctx, next):
    print(f"\n[LOG] Event: {ctx.event_type}")
    await next()

# Handler: Order created
@on("order.created")
async def on_order_created(ctx):
    order = ctx.subject
    amount = ctx.payload["amount"]
    
    await order.set("status", "pending")
    await order.set("amount", amount)
    await order.set("created_at", datetime.now().isoformat())
    
    # Set metadata (no events)
    await order.set_ext("processed_by", "system")
    
    print(f"[Handler] Order created: ${amount}")

# Handler: Validate order (with filter)
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status" and ctx.new_value == "pending")
async def validate_order(ctx):
    order = ctx.subject
    amount = await order.get("amount")
    
    if amount > 0:
        await order.set("status", "validated")
        print(f"[Handler] Order validated")
    else:
        await order.set("status", "rejected", extra={
            "reason": "invalid_amount"
        })

# Handler: Process payment (with filter)
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status" and ctx.new_value == "validated")
async def process_payment(ctx):
    order = ctx.subject
    amount = await order.get("amount")
    
    # Simulate payment processing
    print(f"[Handler] Processing payment: ${amount}")
    await order.set("status", "paid")
    await order.set("paid_at", datetime.now().isoformat())

# Handler: Ship order (with filter)
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status" and ctx.new_value == "paid")
async def ship_order(ctx):
    order = ctx.subject
    
    await order.set("status", "shipped")
    await order.set("shipped_at", datetime.now().isoformat())
    print(f"[Handler] Order shipped!")

# Handler: Rejection (with extra context)
@on("subject-property-changed")
@when(lambda ctx: ctx.property_name == "status" and ctx.new_value == "rejected")
async def handle_rejection(ctx):
    reason = (ctx.extra or {}).get("reason", "unknown")
    print(f"[Handler] Order rejected: {reason}")

# Create and process orders
print("=" * 60)
print("Processing Order 1 (valid):")
print("=" * 60)
order1 = container.subject("order-1")
await emit("order.created", order1, {"amount": 1500})
await asyncio.sleep(0.1)  # Allow handlers to execute

print("\n" + "=" * 60)
print("Processing Order 2 (invalid):")
print("=" * 60)
order2 = container.subject("order-2")
await emit("order.created", order2, {"amount": -100})
await asyncio.sleep(0.1)

# Display final state
print("\n" + "=" * 60)
print("Final State:")
print("=" * 60)
print(f"Order 1: {await order1.dict()}")
print(f"Order 2: {await order2.dict()}")

## Summary

This notebook demonstrated:

✅ **Subjects & Properties** - Dynamic entities with reactive state  
✅ **Lambda Values** - Atomic operations (increment, append, max)  
✅ **Event Handlers** - React to events with `@on` decorator  
✅ **Filters** - Conditional execution with `@when`  
✅ **Middleware** - Cross-cutting concerns (logging, timing)  
✅ **Property Changes** - Automatic `subject-property-changed` events  
✅ **Extra Context** - Pass metadata to handlers  
✅ **Extended Properties** - Metadata without events  
✅ **Storage Backends** - In-memory, Redis, PostgreSQL  
✅ **Complete Example** - Order processing workflow  

### Next Steps

- [Documentation](https://github.com/airspot-dev/krules-framework/tree/main/docs)
- [Quick Start Guide](https://github.com/airspot-dev/krules-framework/blob/main/docs/QUICKSTART.md)
- [Advanced Patterns](https://github.com/airspot-dev/krules-framework/blob/main/docs/ADVANCED_PATTERNS.md)
- [API Reference](https://github.com/airspot-dev/krules-framework/blob/main/docs/API_REFERENCE.md)